# import libraries

In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns 

# Import ONS Data

In [116]:
# Import the energy intensity data for the CPA categories
cpa_energy_intensity = pd.read_excel('./Data/cpa to coicop conversion.xlsx', sheet_name='CPA energy intensity transposed')

# rename the cpa column so it matches the other dataframe
cpa_energy_intensity.rename(columns={'CPA':'CPA Code'}, inplace=True)

# drop description as we don't need it
cpa_energy_intensity.drop('Product', axis=1, inplace=True)

cpa_energy_intensity.head()

,CPA Code,energy intensity
0,CPA_A01,3.464427
1,CPA_A02,1.513570
2,CPA_A03,4.351723
3,CPA_B05,3.005464
4,CPA_B06 & B07,3.386310


In [117]:
# Import the ONS conversion table 
conversion = pd.read_excel('./Data/cpa to coicop conversion.xlsx', sheet_name='COICOP to CPA conversion')

conversion.head()

,COICOP Code,COICOP Description,CPA Code,CPA Description,2019 contribution
0,01.1.1,Bread and cereals,CPA_C106,"Grain mill products, starches and starch products",0.180
1,01.1.1,Bread and cereals,CPA_C107,Bakery and farinaceous products,0.579
2,01.1.1,Bread and cereals,CPA_C108,Other food products,0.241
3,01.1.2,Meat,CPA_C101,Preserved meat and meat products,1.000
4,01.1.3,Fish,CPA_A03,Fish and other fishing products; aquaculture p...,0.157


# Calculate energy intensity for COICOP categories

In [118]:
# merge the dataframes so we can get the energy intensity data into the conversion table
df = pd.merge(conversion, cpa_energy_intensity, on='CPA Code', how='left')

# Rename columns for clarity
df.rename(columns={'energy intensity':'Energy Intensity of CPA category', '2019 contribution':'CPA contribution to COICOP (2019)'}, inplace=True)

df.head()

# add another column to calculate the energy intensity contribition from each CPA
df['Contribution to energy intensity from CPA'] = df['CPA contribution to COICOP (2019)'] * df['Energy Intensity of CPA category']

df.head(100)

,COICOP Code,COICOP Description,CPA Code,CPA Description,CPA contribution to COICOP (2019),Energy Intensity of CPA category,Contribution to energy intensity from CPA
0,01.1.1,Bread and cereals,CPA_C106,"Grain mill products, starches and starch products",0.180,2.045947,0.368270
1,01.1.1,Bread and cereals,CPA_C107,Bakery and farinaceous products,0.579,2.583945,1.496104
2,01.1.1,Bread and cereals,CPA_C108,Other food products,0.241,1.964081,0.473343
3,01.1.2,Meat,CPA_C101,Preserved meat and meat products,1.000,1.832863,1.832863
4,01.1.3,Fish,CPA_A03,Fish and other fishing products; aquaculture p...,0.157,4.351723,0.683221
...,...,...,...,...,...,...,...
95,06.1.1,Pharmaceutical products,CPA_O84,Public administration and defence services; co...,0.045,1.056495,0.047542
96,06.1.1,Pharmaceutical products,CPA_C21,Basic pharmaceutical products and pharmaceutic...,0.955,0.884737,0.844924
97,06.1.2,Other medical products,CPA_C22,Rubber and plastic products,0.700,2.763877,1.934714
98,06.1.2,Other medical products,CPA_C32,Other manufactured goods,0.300,1.313425,0.394028


In [119]:
# compute the sum of the different energy intensity contributions from the various CPA categories 
# in order to get the total energy intensities of the COICOP categories

result = df.groupby('COICOP Code')['Contribution to energy intensity from CPA'].sum()

# The result is a pandas series so lets convert in into a dataframe
coicop_energy_intensity = result.to_frame()

# rename column
coicop_energy_intensity.rename(columns={'Contribution to energy intensity from CPA':'Energy Intensity'}, inplace=True)

# turn index into a column so we can access the data
coicop_energy_intensity = coicop_energy_intensity.reset_index()

coicop_energy_intensity.head(50)

,COICOP Code,Energy Intensity
0,12.4,0.094999
1,12.7,0.884269
2,01.1.1,2.337718
3,01.1.2,1.832863
4,01.1.3,2.154789
5,01.1.4,1.932987
6,01.1.5,2.596474
7,01.1.6,3.394175
8,01.1.7,2.610186
9,01.1.8,1.925108


In [120]:
# Find the average energy intensity for each coicop category
coicop_category_intensity = coicop_energy_intensity.groupby(coicop_energy_intensity['COICOP Code'].str[:2]).mean()

coicop_category_intensity = coicop_category_intensity.reset_index()

print(coicop_category_intensity)


   COICOP Code  Energy Intensity
0           01          2.212783
1           02          0.176947
2           03          1.230792
3           04         15.819519
4           05          1.477161
5           06          1.153774
6           07          9.161693
7           08          1.383862
8           09          1.223634
9           10          0.904353
10          11          2.281791
11          12          0.574229


/var/folders/t7/n_c5tq1n0h985pbhlhf2w0c80000gn/T/ipykernel_26027/1377313516.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  coicop_category_intensity = coicop_energy_intensity.groupby(coicop_energy_intensity['COICOP Code'].str[:2]).mean()


In [121]:
# add the list of coicop categories

coicop_categories = ['Food and non-alcoholic beverages', 
                     'Alcoholic beverages, tobacco and narcotics', 
                     'Clothing and footwear',
                     'Housing, water, electricity, gas and other fuels',
                     'Furnishings, household equipment and routine household maintenance',
                     'Health',
                     'Transport',
                     'Communication',
                     'Recreation and culture',
                     'Education',
                     'Restaurants and hotels',
                     'Miscellaneous goods and services'
]

In [122]:
# add category names to dataframe
coicop_category_intensity['COICOP Category names'] = coicop_categories

# move columns around
e = coicop_category_intensity.pop('Energy Intensity')
coicop_category_intensity['Energy Intensity'] = e

coicop_category_intensity.head(20)

,COICOP Code,COICOP Category names,Energy Intensity
0,01,Food and non-alcoholic beverages,2.212783
1,02,"Alcoholic beverages, tobacco and narcotics",0.176947
2,03,Clothing and footwear,1.230792
3,04,"Housing, water, electricity, gas and other fuels",15.819519
4,05,"Furnishings, household equipment and routine h...",1.477161
5,06,Health,1.153774
6,07,Transport,9.161693
7,08,Communication,1.383862
8,09,Recreation and culture,1.223634
9,10,Education,0.904353
